In [2]:
import os

In [3]:
parent_dir = "../data"

In [17]:

all_files = []
for root, dirs, files in os.walk(parent_dir):
    for file in files:
        all_files.append(os.path.join(root, file))


In [5]:
from markitdown import MarkItDown
from openaivec import pandas_ext
from openai import AzureOpenAI, AsyncAzureOpenAI
import pandas as pd

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    base_url=os.getenv("AZURE_OPENAI_BASE_URL"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

async_client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    base_url=os.getenv("AZURE_OPENAI_BASE_URL"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

md = MarkItDown(llm_client=client, llm_model="gpt-5-mini")

pandas_ext.set_client(client)
pandas_ext.set_async_client(async_client)
pandas_ext.set_responses_model("gpt-5-mini")


client.responses.create(model="gpt-5-mini", input="Hello, world!", reasoning={"effort": "minimal"}).output_text



'Hello! 👋 How can I help you today?'

In [6]:
sources = pd.DataFrame({"file_path": all_files})

In [7]:
docs = (
    sources
    .assign(
        parsed=lambda df: df.file_path
        .map(md.convert)
        .map(lambda x: x.text_content)
    )
)

In [8]:
for file_path, parsed in zip(docs.file_path, docs.parsed):
    with open(f"{str(file_path)}.md".replace("data", "parsed"), "w") as f:
        try:
            f.write(parsed)
        except:
            f.write("Error in writing parsed content.")

In [9]:
from pydantic import BaseModel, Field

class Question(BaseModel):
    question: str = Field(
        ...,
        description=(
            "FAQの質問文。必ず質問文単体で意味が完全に成立するように作ること。"
            "指示語（「これ」「それ」「あれ」「このプログラム」「この仕組み」「上記」など）は一切使用してはならない。"
            "質問文だけ読んだ読者が文脈なしで理解できるように、必要な固有名詞・対象名・機能名・手順名を必ず質問文中に明示すること。"
            "質問文はFAQとして自然な形式（「〜とは何ですか？」「〜を行うにはどうすればよいですか？」など）で作成する。"
            "質問文を生成する前に以下の手順を内部で必ず実行する："
            "1) セクション内容から固有名詞・重要概念・機能・手順を抽出する。"
            "2) 抽出した対象を明示的に含む質問文を構成する。"
            "3) 質問文単体で意味が通じるか確認し、不足する固有名詞を補完する。"
            "4) 質問内容が他のFAQと重複しないように注意する。"
        )
    )
    answer: str = Field(
        ...,
        description=(
            "FAQの回答文。質問文に対して直接的かつ明確に答えること。"
            "不要な文脈説明を避け、質問に対応する情報を中心に構成する。"
            "必要に応じて追加情報を補足してもよいが、質問に回答していることが最優先。"
        )
    )

class Section(BaseModel):
    title: str = Field(..., description="セクションのタイトル。")
    content: str = Field(..., description="セクションの内容。FAQ生成の元となる説明文。")
    faqs: list[Question] = Field(
        ...,
        description=(
            "このセクションに関連するFAQのリスト。"
            "1セクションにつき最低2件以上のFAQを生成すること。"
            "同じ内容を別の言い回しで繰り返した質問を作らないこと。"
        )
    )

class Document(BaseModel):
    sections: list[Section] = Field(..., description="ドキュメント全体のセクションのリスト。")

In [12]:
instructions = """
あなたは与えられた，PowerPointファイルをMarkdownにパースしたドキュメントから高品質な FAQ を生成する専門アシスタントです。
以下の Pydantic モデルに厳密に従い、構造化された JSON を出力してください。

出力は次の JSON スキーマに準拠すること：

{
  "sections": [
    {
      "title": "string",
      "content": "string",
      "faqs": [
        {
          "question": "string",
          "answer": "string"
        }
      ]
    }
  ]
}

## FAQ 生成ルール（必ず遵守すること）

### 1. 質問文は単体で意味が成立すること
- 指示語（「これ」「それ」「あれ」「このプログラム」など）禁止
- 文脈依存の表現禁止（「上記」「以下」など）
- 必要な固有名詞、対象名称、機能名、制度名を質問文内に明記する

### 2. 質問文の自然さ
- FAQ の一般的形式で書くこと
  - 「〜とは何ですか？」
  - 「〜を利用するにはどうすればよいですか？」
  - 「〜の要件は何ですか？」
- 質問文は読み手が文脈ゼロでも理解できること

### 3. 質問生成の内部手順（モデルは内部で必ず実行すること）
1. セクションから固有名詞、概念、機能、要件、手順を抽出する  
2. それぞれを中心に質問文を構成する  
3. 質問文単体で理解できるかチェックし、固有名詞を補完する  
4. 他の質問と内容が重複しないことを確認する  

### 4. 回答文のルール
- 質問に直接答えることが最優先
- 不要な文脈説明や脱線を避ける
- 必要なら補足情報をつけてもよいが、質問に答えることが主目的

### 5. 各セクションの FAQ 数
- 1 セクションあたり最低 2 件以上生成する
- 重複内容は作らない

## 出力フォーマット
- **必ず JSON のみ**を出力する
- JSON 以外の文章や説明文は禁止
- 上記スキーマに完全に一致する形式で出力する

以上のルールに基づき、入力ドキュメントから FAQ を生成してください。

"""

docs["document"] = await (
    docs.parsed.aio.responses(
        instructions=instructions,
        response_format=Document,
        reasoning={"effort": "minimal"},
        batch_size=1,
        max_concurrency=5,
    )
)

Processing batches:   0%|          | 0/4 [00:00<?, ?item/s]

In [16]:
(
    docs
    .assign(
        file_path=lambda df: df.file_path.map(lambda x: x.split("/")[-1])
    )
    .drop(columns=["parsed"])
    .ai.extract("document")
    .explode("document_sections")
    .ai.extract("document_sections")
    .explode("document_sections_faqs")
    .ai.extract("document_sections_faqs")
    .reset_index(drop=True)
    .to_csv("../parsed/faqs.csv", index=False)
)